## Testes da API

Nesse notebook são feitos alguns testes com a API do Olho Vivo

In [1]:
import pandas as pd
from requests import Session
from typing import Optional

In [2]:
class RequestMaker:

    def __init__(self, method:str, host:str, version:Optional[str]=None)->None:

        self.method = method
        self.host = host
        self.version = version
        self.session = Session()

        self.url_base = self.build_base_url()

    def build_base_url(self):

        url = f'{self.method}://{self.host}'

        if self.version:
            url = url+f'/v{self.version}'

        return url

    def build_params_str(self, params:dict)->str:

        params = [f"{key}={value}" for key, value in params.items()]
        params = '&'.join(params)
        params = f'?{params}'

        return params

    def build_request_url(self, endpoint:Optional[str]=None, params:Optional[dict]=None)->str:

        url = self.url_base
        if endpoint:
            url = self.url_base + '/' + endpoint.strip('/')
        
        if params:
            params_str = self.build_params_str(params)
            url = url + params_str
        
        return url
               
    def post(self,endpoint:Optional[str]=None, params:Optional[dict]=None, data:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo post em: {url}. Dados: {data}')
        with self.session.post(url, data=data) as r:
            if as_json:
                return r.json()
            return r.text
        
    def get(self, endpoint:Optional[str]=None, params:Optional[dict]=None, as_json:bool=True)->dict:

        url = self.build_request_url(endpoint, params)
        print(f'Fazendo get em {url}.')
        with self.session.get(url) as r:
            if as_json:
                return r.json()
            return r.text



class APIOlhoVivoClient:

    HOST='api.olhovivo.sptrans.com.br'
    METHOD='http'
    VERSION='2.1'

    def __init__(self, token:str):

        self.request = RequestMaker(self.METHOD, self.HOST, self.VERSION)
        self.token=token

        self.autenticar()

    def autenticar(self)->None:

        endpoint = 'Login/Autenticar'
        params = {'token' : self.token}

        #a api nao retorna um json valido
        resp = self.request.post(endpoint, params, as_json=False)
        success = resp=='true'
        if not success:
            raise RuntimeError(f'Falha ao atenticar: {resp}')
        print('Autenticado com sucesso.')

    @property
    def posicao_atual_onibus(self):

        endpoint = 'Posicao'
        return self.request.get(endpoint)

In [3]:
token = '7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0'

In [4]:
api = APIOlhoVivoClient(token)

Fazendo post em: http://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=7c54257f14ee2043182985954596f0b6c6cda242e6447f817338f45f29a593c0. Dados: None
Autenticado com sucesso.


In [5]:
posicoes = []

#espera = 30
for i in range(1):
    posicao_atual = api.posicao_atual_onibus
    posicoes.append(posicao_atual)
    #time.sleep(espera)

posicoes

Fazendo get em http://api.olhovivo.sptrans.com.br/v2.1/Posicao.


[{'hr': '11:32',
  'l': [{'c': '978A-10',
    'cl': 1369,
    'sl': 1,
    'lt0': 'METRÔ BARRA FUNDA',
    'lt1': 'TERM. CACHOEIRINHA',
    'qv': 7,
    'vs': [{'p': 12475,
      'a': True,
      'ta': '2025-09-18T14:31:59Z',
      'py': -23.4881395,
      'px': -46.68366975000001,
      'sv': None,
      'is': None},
     {'p': 12594,
      'a': True,
      'ta': '2025-09-18T14:31:55Z',
      'py': -23.510803,
      'px': -46.690124499999996,
      'sv': None,
      'is': None},
     {'p': 12469,
      'a': True,
      'ta': '2025-09-18T14:31:33Z',
      'py': -23.508825,
      'px': -46.6896915,
      'sv': None,
      'is': None},
     {'p': 12467,
      'a': True,
      'ta': '2025-09-18T14:31:28Z',
      'py': -23.478749999999998,
      'px': -46.673298,
      'sv': None,
      'is': None},
     {'p': 12433,
      'a': True,
      'ta': '2025-09-18T14:31:30Z',
      'py': -23.519757,
      'px': -46.6749715,
      'sv': None,
      'is': None},
     {'p': 12476,
      'a': True,
 

In [6]:
len(posicoes)

1

In [7]:
rows = []

for h in posicoes:
    if h:
        for l in h['l']:

            for veiculo in l['vs']:

                row = {
                    'linha': l['c'],
                    'id_onibus': veiculo['p'],
                    'timestamp': veiculo['ta'],
                    'lat': veiculo['py'],
                    'lon': veiculo['px'] 
                }

                rows.append(row)

df_posicoes = pd.DataFrame(rows)

Conversão de horário

In [8]:
df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,978A-10,12475,2025-09-18T14:31:59Z,-23.488139,-46.683670
1,978A-10,12594,2025-09-18T14:31:55Z,-23.510803,-46.690124
2,978A-10,12469,2025-09-18T14:31:33Z,-23.508825,-46.689692
3,978A-10,12467,2025-09-18T14:31:28Z,-23.478750,-46.673298
4,978A-10,12433,2025-09-18T14:31:30Z,-23.519757,-46.674971
...,...,...,...,...,...
10298,9032-21,16509,2025-09-18T14:31:22Z,-23.474238,-46.669689
10299,6058-10,66104,2025-09-18T14:31:50Z,-23.772083,-46.700250
10300,4010-10,48815,2025-09-18T14:31:47Z,-23.582402,-46.497104
10301,4008-21,48273,2025-09-18T14:31:20Z,-23.579354,-46.432372


In [9]:
df_posicoes['timestamp'] = pd.to_datetime(df_posicoes['timestamp'])

df_posicoes['timestamp'] = df_posicoes['timestamp'].dt.tz_convert("America/Sao_Paulo")

df_posicoes

,linha,id_onibus,timestamp,lat,lon
0,978A-10,12475,2025-09-18 11:31:59-03:00,-23.488139,-46.683670
1,978A-10,12594,2025-09-18 11:31:55-03:00,-23.510803,-46.690124
2,978A-10,12469,2025-09-18 11:31:33-03:00,-23.508825,-46.689692
3,978A-10,12467,2025-09-18 11:31:28-03:00,-23.478750,-46.673298
4,978A-10,12433,2025-09-18 11:31:30-03:00,-23.519757,-46.674971
...,...,...,...,...,...
10298,9032-21,16509,2025-09-18 11:31:22-03:00,-23.474238,-46.669689
10299,6058-10,66104,2025-09-18 11:31:50-03:00,-23.772083,-46.700250
10300,4010-10,48815,2025-09-18 11:31:47-03:00,-23.582402,-46.497104
10301,4008-21,48273,2025-09-18 11:31:20-03:00,-23.579354,-46.432372


In [10]:
horas = df_posicoes["timestamp"].dt.hour

horas_unicas = horas.unique()
horas_unicas.sort()

print(horas_unicas)

[10 11]


In [15]:
onze_horas = df_posicoes[df_posicoes["timestamp"].dt.hour == 10]

onze_horas

,linha,id_onibus,timestamp,lat,lon
6830,1745-10,22918,2025-09-18 10:27:42-03:00,-23.473012,-46.665725


Ônibus duplicados

In [12]:
duplicados = (df_posicoes.groupby(["id_onibus", "timestamp"]).filter(lambda x: len(x) > 1).sort_values(["id_onibus", "timestamp"]))

duplicados

,linha,id_onibus,timestamp,lat,lon
243,930P-10,32017,2025-09-18 11:31:31-03:00,-23.566185,-46.702046
4519,263C-10,32017,2025-09-18 11:31:31-03:00,-23.483906,-46.415997
